# Batch Inference with Champion Model
Load the champion model from Unity Catalog, run batch inference on new gold features, and write predictions to a Delta table.

In [ ]:
# Load the champion model from Unity Catalog, run batch inference, and write predictions to Delta table
import mlflow
import pandas as pd

gold_path = "/Volumes/your_catalog/your_schema/energy_price_gold_features"
prediction_path = "/Volumes/your_catalog/your_schema/energy_price_predictions"
model_uc_path = "models:/your_catalog/your_schema/energy_price_xgb@champion"

mlflow.set_registry_uri("databricks-uc")
champion_model = mlflow.pyfunc.load_model(model_uc_path)

gold_pdf = spark.read.format("delta").load(gold_path).toPandas()
feature_cols = [c for c in gold_pdf.columns if c not in ["start_time", "end_time", "price_eur_mwh"]]
X_pred = gold_pdf[feature_cols]
pred_df = gold_pdf[["start_time"]].copy()
pred_df["prediction"] = champion_model.predict(X_pred)

pred_sdf = spark.createDataFrame(pred_df)
pred_sdf.write.format("delta").mode("overwrite").save(prediction_path)
print(f"Predictions written to {prediction_path}")